In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
sns.set(style="whitegrid")
%matplotlib inline
import datetime, warnings, scipy 
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import ConnectionPatch
import nltk
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
#import spacy
from sklearn.model_selection import train_test_split
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from collections import OrderedDict
from matplotlib.gridspec import GridSpec
#from mpl_toolkits.basemap import Basemap
from scipy.optimize import curve_fit
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
pd.options.display.max_columns = 50
warnings.filterwarnings("ignore")
from glob import glob

In [ ]:
tr_text=pd.read_csv("C:/Users/lohitr/Desktop/train_2kmZucJ.csv")
ts_text=pd.read_csv("C:/Users/lohitr/Desktop/test_oJQbWVk.csv")

In [ ]:
tr_text.shape , ts_text.shape

In [ ]:
tr_text.head()

In [ ]:
ts_text.head()

In [ ]:
tweet= tr_text.tweet
print(tweet)

In [ ]:
def text_data(text):
    
    text = re.sub(r'@[A-Za-z0-9]+','',text)
    text =re.sub(r'#','',text)
    text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)
    text = re.sub(r'www.[^ ]+', '', text)
    text = re.sub(r'[a-zA-Z0-9]*www[a-zA-Z0-9]*com[a-zA-Z0-9]*', '', text)  
    text = re.sub(r'[^a-zA-Z]', ' ', text)   
    text = [token for token in text.split() if len(token) > 2]
    text = ' '.join(text)
    #text= re.sub(r'https?\/\/\S+','',text)
    
    return text

tr_text['tweet']=tr_text['tweet'].apply(text_data)

In [ ]:
tr_text['tweet']

In [ ]:
from textblob  import TextBlob

In [ ]:
### Create a function to get the subjectivity(opinion of text) 
def gets_subjet(text):
    return TextBlob(text).sentiment.subjectivity

### Create a function to know polarity (how much positive and negative in columns)
def gets_subjet(text):
    return TextBlob(text).sentiment.polarity


tr_text['subjectivity']= tr_text['tweet'].apply(gets_subjet)
tr_text['polarity']= tr_text['tweet'].apply(gets_subjet)


In [ ]:
tr_text

In [ ]:
# Plot the word cloud
Words_View = ''.join([y for y in tr_text['tweet']])
Cloud = WordCloud(width=500,height=300,random_state=21,max_font_size=117).generate(Words_View)
plt.imshow(Cloud,interpolation="bilinear")
plt.axis('Off')
plt.show()

In [ ]:
# Create a function to positive , neutral & negative analysis
def getAnalysis(score):
    if score <0:
        return 'Negative'
    elif score ==0:
        return 'Neutral'
    else:
        return 'Positive'
    
tr_text['Analysis']=tr_text['polarity'].apply(getAnalysis)

In [ ]:
tr_text

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(tr_text['tweet'], tr_text['Analysis'], test_size=0.25, stratify=tr_text['Analysis'], 
                                                random_state=1)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(analyzer='word', stop_words=None, token_pattern='(?u)\\b\\w\\w+\\b', ngram_range=(1, 1), max_df=1.0, 
                             min_df=3, max_features=None, binary=False, norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False)

In [ ]:
X_train_tfidf = tfidf_vect.fit_transform(X_train)
X_cv_tfidf = tfidf_vect.transform(X_cv)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(loss='log', max_iter=200, random_state=0, class_weight='balanced')
ovr = OneVsRestClassifier(sgd)
ovr.fit(X_train_tfidf, y_train)
y_pred_class = ovr.predict(X_cv_tfidf)
print('f1_score       :', f1_score(y_cv, y_pred_class, average='macro'))
print('accuracy score :', accuracy_score(y_cv, y_pred_class))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(analyzer='word', stop_words=None, token_pattern='(?u)\\b\\w\\w+\\b', ngram_range=(1, 1), max_df=1.0, 
                             min_df=3, max_features=None, binary=False, norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False)

In [ ]:
ts_text

In [ ]:
full_text = list(tr_text['tweet'].values) + list(ts_text['tweet'].values)
tfidf_vect.fit(full_text)

X_train_tfidf = tfidf_vect.transform(tr_text['tweet'])
X_test_tfidf = tfidf_vect.transform(ts_text['tweet'])

y_train = tr_text['Analysis']

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(loss='log', max_iter=200, random_state=0, class_weight='balanced')
ovr = OneVsRestClassifier(sgd)
ovr.fit(X_train_tfidf, y_train)
y_pred_class = ovr.predict(X_test_tfidf)
y_pred_class

In [ ]:
ts_text

In [ ]:
ts_text['label'] = y_pred_class
label_encoder = preprocessing.LabelEncoder()
ts_text['label'] = label_encoder.fit_transform(ts_text['label'])
#ts_text.drop(['tweet'], axis=1,inplace=True)
#ts_text.head()

In [ ]:
ts_text['label'].value_counts()

In [ ]:
ts_text.to_csv('C:/Users/lohitr/Desktop/submission.csv', index=False)